# read

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/train_df.csv', encoding='cp949')
test  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/test_df.csv', encoding='cp949')
sub   = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/sample_submission.csv', encoding='cp949')

In [2]:
train.shape, test.shape, sub.shape

((32000, 6), (4640, 5), (4640, 2))

In [3]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


# 앞 10자리만 남김

In [4]:
for df in [train, test]:
    df['round_send'] = df['SEND_SPG_INNB'].round(-10)
    df['round_rec']  = df['REC_SPG_INNB'].round(-10)

In [5]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,round_send,round_rec
0,0,1129000014045300,5011000220046300,패션의류,상의,3,1129000000000000,5011000000000000
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3,1135000000000000,5011000000000000
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9,1135000000000000,5011000000000000
3,3,1154500002014200,5011000315087400,식품,농산물,10,1154500000000000,5011000000000000
4,4,1165000021008300,5011000177051200,식품,가공식품,3,1165000000000000,5011000000000000


# 격자공간 범주화

In [6]:
send_list = list(set(train['round_send'].to_list()))
rec_list  = list(set(train['round_rec'].to_list()))

sum_list = list(set(send_list + rec_list))
len(sum_list)

251

In [7]:
encode_list = []
for i in range(len(sum_list)):
    encode_list.append([sum_list[i], str(i)])

encode_dict = dict(encode_list)

In [8]:
for df in [train, test]:
    df['send_dict'] = df['round_send'].map(encode_dict)
    df['rec_dict'] = df['round_rec'].map(encode_dict)

# get_dummies

In [9]:
train = pd.get_dummies(train)
test  = pd.get_dummies(test)

In [10]:
features = train.columns[6:]
train[features]

,DL_GD_LCLS_NM_디지털/가전,DL_GD_LCLS_NM_생활/건강,DL_GD_LCLS_NM_식품,DL_GD_LCLS_NM_여행/문화,DL_GD_LCLS_NM_패션의류,DL_GD_LCLS_NM_화장품/미용,DL_GD_MCLS_NM_가공식품,DL_GD_MCLS_NM_건강식품,DL_GD_MCLS_NM_건강용품,DL_GD_MCLS_NM_과자,DL_GD_MCLS_NM_기타디지털/가전,DL_GD_MCLS_NM_기타식품,DL_GD_MCLS_NM_기타패션의류,DL_GD_MCLS_NM_냉동/간편조리식품,DL_GD_MCLS_NM_농산물,DL_GD_MCLS_NM_문구/사무용품,DL_GD_MCLS_NM_문화컨텐츠,DL_GD_MCLS_NM_반려동물,DL_GD_MCLS_NM_상의,DL_GD_MCLS_NM_생활용품,DL_GD_MCLS_NM_수산,DL_GD_MCLS_NM_스킨케어,DL_GD_MCLS_NM_음료,DL_GD_MCLS_NM_음반,DL_GD_MCLS_NM_주방용품,DL_GD_MCLS_NM_축산,send_dict_0,send_dict_1,send_dict_10,send_dict_100,send_dict_101,send_dict_102,send_dict_103,send_dict_104,send_dict_106,send_dict_107,send_dict_109,send_dict_11,send_dict_110,send_dict_111,...,rec_dict_63,rec_dict_64,rec_dict_65,rec_dict_66,rec_dict_67,rec_dict_68,rec_dict_69,rec_dict_7,rec_dict_70,rec_dict_71,rec_dict_72,rec_dict_73,rec_dict_74,rec_dict_75,rec_dict_76,rec_dict_77,rec_dict_78,rec_dict_79,rec_dict_8,rec_dict_80,rec_dict_81,rec_dict_82,rec_dict_83,rec_dict_84,rec_dict_85,rec_dict_86,rec_dict_87,rec_dict_88,rec_dict_89,rec_dict_9,rec_dict_90,rec_dict_91,rec_dict_92,rec_dict_93,rec_dict_94,rec_dict_95,rec_dict_96,rec_dict_97,rec_dict_98,rec_dict_99
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31996,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31997,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31998,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
test

,index,SEND_SPG_INNB,REC_SPG_INNB,round_send,round_rec,DL_GD_LCLS_NM_디지털/가전,DL_GD_LCLS_NM_생활/건강,DL_GD_LCLS_NM_식품,DL_GD_LCLS_NM_여행/문화,DL_GD_LCLS_NM_패션의류,DL_GD_LCLS_NM_화장품/미용,DL_GD_MCLS_NM_가공식품,DL_GD_MCLS_NM_건강식품,DL_GD_MCLS_NM_건강용품,DL_GD_MCLS_NM_과자,DL_GD_MCLS_NM_기타디지털/가전,DL_GD_MCLS_NM_기타식품,DL_GD_MCLS_NM_기타패션의류,DL_GD_MCLS_NM_냉동/간편조리식품,DL_GD_MCLS_NM_농산물,DL_GD_MCLS_NM_문구/사무용품,DL_GD_MCLS_NM_문화컨텐츠,DL_GD_MCLS_NM_반려동물,DL_GD_MCLS_NM_상의,DL_GD_MCLS_NM_생활용품,DL_GD_MCLS_NM_수산,DL_GD_MCLS_NM_스킨케어,DL_GD_MCLS_NM_음료,DL_GD_MCLS_NM_음반,DL_GD_MCLS_NM_주방용품,DL_GD_MCLS_NM_축산,send_dict_0,send_dict_1,send_dict_10,send_dict_101,send_dict_102,send_dict_103,send_dict_104,send_dict_109,send_dict_110,...,rec_dict_63,rec_dict_64,rec_dict_65,rec_dict_66,rec_dict_67,rec_dict_68,rec_dict_69,rec_dict_7,rec_dict_70,rec_dict_71,rec_dict_72,rec_dict_73,rec_dict_74,rec_dict_75,rec_dict_76,rec_dict_77,rec_dict_78,rec_dict_79,rec_dict_8,rec_dict_80,rec_dict_81,rec_dict_82,rec_dict_83,rec_dict_84,rec_dict_85,rec_dict_86,rec_dict_87,rec_dict_88,rec_dict_89,rec_dict_9,rec_dict_90,rec_dict_91,rec_dict_92,rec_dict_93,rec_dict_94,rec_dict_95,rec_dict_96,rec_dict_97,rec_dict_98,rec_dict_99
0,32000,5013000043028400,1165000021097200,5013000000000000,1165000000000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,32001,5013000044016100,1154500002066400,5013000000000000,1154500000000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,32002,5013000205030200,4139000102013200,5013000000000000,4139000000000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,32003,5013000205030200,4221000040093400,5013000000000000,4221000000000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,32004,5013000268011400,2726000004017100,5013000000000000,2726000000000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,36635,5013000858004400,4725000719072200,5013000000000000,4725000000000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4636,36636,5013000870018300,2826000106075300,5013000000000000,2826000000000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4637,36637,5013000897086300,4311100034004300,5013000000000000,4311100000000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4638,36638,5013000902065100,4145000013011200,5013000000000000,4145000000000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
# train에는 있지만 test에는 없는 컬럼 추가
from tqdm import tqdm

for f in tqdm(features):
    if f in list(test.columns):
        pass
    else:
        test[f] = int(0)

100%|██████████| 511/511 [00:00<00:00, 7667.09it/s]


In [13]:
test[features]

,DL_GD_LCLS_NM_디지털/가전,DL_GD_LCLS_NM_생활/건강,DL_GD_LCLS_NM_식품,DL_GD_LCLS_NM_여행/문화,DL_GD_LCLS_NM_패션의류,DL_GD_LCLS_NM_화장품/미용,DL_GD_MCLS_NM_가공식품,DL_GD_MCLS_NM_건강식품,DL_GD_MCLS_NM_건강용품,DL_GD_MCLS_NM_과자,DL_GD_MCLS_NM_기타디지털/가전,DL_GD_MCLS_NM_기타식품,DL_GD_MCLS_NM_기타패션의류,DL_GD_MCLS_NM_냉동/간편조리식품,DL_GD_MCLS_NM_농산물,DL_GD_MCLS_NM_문구/사무용품,DL_GD_MCLS_NM_문화컨텐츠,DL_GD_MCLS_NM_반려동물,DL_GD_MCLS_NM_상의,DL_GD_MCLS_NM_생활용품,DL_GD_MCLS_NM_수산,DL_GD_MCLS_NM_스킨케어,DL_GD_MCLS_NM_음료,DL_GD_MCLS_NM_음반,DL_GD_MCLS_NM_주방용품,DL_GD_MCLS_NM_축산,send_dict_0,send_dict_1,send_dict_10,send_dict_100,send_dict_101,send_dict_102,send_dict_103,send_dict_104,send_dict_106,send_dict_107,send_dict_109,send_dict_11,send_dict_110,send_dict_111,...,rec_dict_63,rec_dict_64,rec_dict_65,rec_dict_66,rec_dict_67,rec_dict_68,rec_dict_69,rec_dict_7,rec_dict_70,rec_dict_71,rec_dict_72,rec_dict_73,rec_dict_74,rec_dict_75,rec_dict_76,rec_dict_77,rec_dict_78,rec_dict_79,rec_dict_8,rec_dict_80,rec_dict_81,rec_dict_82,rec_dict_83,rec_dict_84,rec_dict_85,rec_dict_86,rec_dict_87,rec_dict_88,rec_dict_89,rec_dict_9,rec_dict_90,rec_dict_91,rec_dict_92,rec_dict_93,rec_dict_94,rec_dict_95,rec_dict_96,rec_dict_97,rec_dict_98,rec_dict_99
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4636,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4637,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4638,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# modeling

In [14]:
X = train[features]
y = train['INVC_CONT']

In [15]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_ds = lgb.Dataset(X_train, label=y_train)
test_ds  = lgb.Dataset(X_test, label=y_test)

params = { 
          'learning_rate': 0.082, 
          'max_depth': 5, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 50, 
          'feature_fraction': 0.8, 
          'bagging_fraction': 0.6, 
          'bagging_freq': 2, 
          'min_data_in_leaf':2,
}

model = lgb.train(params, train_ds, 1500, test_ds, verbose_eval=100, early_stopping_rounds=200)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's l2: 30.9619
[200]	valid_0's l2: 31.338
Early stopping, best iteration is:
[70]	valid_0's l2: 30.8213


In [16]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, model.predict(X_test), squared=False)

5.551697100636408

# sub

In [17]:
test.columns

Index(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB', 'round_send', 'round_rec',
       'DL_GD_LCLS_NM_디지털/가전', 'DL_GD_LCLS_NM_생활/건강', 'DL_GD_LCLS_NM_식품',
       'DL_GD_LCLS_NM_여행/문화', 'DL_GD_LCLS_NM_패션의류',
       ...
       'send_dict_83', 'send_dict_87', 'send_dict_92', 'send_dict_94',
       'send_dict_95', 'send_dict_98', 'rec_dict_141', 'rec_dict_46',
       'rec_dict_47', 'rec_dict_51'],
      dtype='object', length=519)

In [18]:
sub['INVC_CONT'] = model.predict(test[features])
sub

,index,INVC_CONT
0,32000,4.320855
1,32001,4.320855
2,32002,4.320855
3,32003,4.320855
4,32004,4.320855
...,...,...
4635,36635,9.113954
4636,36636,4.320855
4637,36637,4.320855
4638,36638,4.320855
